<a href="https://colab.research.google.com/github/FatemehRajabi/gold-usd-prediction/blob/main/Gold_USD_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install yfinance scikit-learn


In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Get today's date
today = datetime.date.today()

# Load Gold ETF (GLD) data from Yahoo Finance
data = yf.download('GLD', start='2015-01-01', end=today)

# Flatten MultiIndex
data.columns = data.columns.get_level_values(0)
data.columns.name = None

# Keep only relevant columns
data = data[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

data.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2015-01-02,112.489998,114.800003,112.320000,114.080002,7109600
2015-01-05,114.779999,116.000000,114.730003,115.800003,8177400
2015-01-06,116.220001,117.500000,115.800003,117.120003,11238300
2015-01-07,116.470001,116.879997,116.169998,116.430000,6434200
2015-01-08,116.449997,116.870003,115.849998,115.940002,7033700


In [15]:
# Simple Moving Averages
data['SMA_10'] = data['Close'].rolling(window=10).mean()
data['SMA_30'] = data['Close'].rolling(window=30).mean()

# Relative Strength Index (RSI)
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI_14'] = 100 - (100 / (1 + rs))

# Drop rows with NaNs from indicators
data.dropna(inplace=True)

data.head()

,Open,High,Low,Close,Volume,SMA_10,SMA_30,RSI_14
Date,,,,,,,,
2015-02-13,118.050003,118.580002,117.790001,117.980003,4108900,119.551000,120.106667,35.742753
2015-02-17,116.400002,116.540001,115.580002,116.010002,7225200,118.910001,120.171001,26.861544
2015-02-18,115.989998,116.529999,114.989998,116.339996,8336000,118.439000,120.189000,29.748256
2015-02-19,116.400002,116.540001,115.739998,115.940002,6528700,117.875000,120.149667,34.165550
2015-02-20,116.099998,116.459999,115.050003,115.279999,6681700,117.224000,120.111334,19.029545


In [16]:
# Normalize all features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
print("Scaled shape:", scaled_data.shape)

Scaled shape: (2546, 8)


In [17]:
# Create sequences (60 timesteps back)
sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])  # shape: (60, 8)
    y.append(scaled_data[i, 3])  # predict the Close price (index 3)

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)  # (samples, 60, 8)
print("y shape:", y.shape)  # (samples,)

X shape: (2486, 60, 8)
y shape: (2486,)
